In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'D:\\Projects\\chicken_disease_classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config


In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [8]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-05-29 16:43:23,493] {D:\Projects\chicken_disease_classification\src\cnnClassifier\utils\common.py:31} INFO - yaml file: config\config.yaml loaded successfully]
[2024-05-29 16:43:23,494] {D:\Projects\chicken_disease_classification\src\cnnClassifier\utils\common.py:31} INFO - yaml file: params.yaml loaded successfully]
[2024-05-29 16:43:23,496] {D:\Projects\chicken_disease_classification\src\cnnClassifier\utils\common.py:50} INFO - created directory at: artifacts]
[2024-05-29 16:43:23,497] {D:\Projects\chicken_disease_classification\src\cnnClassifier\utils\common.py:50} INFO - created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-05-29 16:43:23,498] {D:\Projects\chicken_disease_classification\src\cnnClassifier\utils\common.py:50} INFO - created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
